<a href="https://colab.research.google.com/github/thevedantt/SarkasamAI/blob/main/SarcasamDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import google.generativeai as genai
import warnings
warnings.filterwarnings('ignore')

class SarcasmDetector:
    """Professional Sarcasm Detection System using Machine Learning and AI"""

    def __init__(self, api_key):
        """Initialize the sarcasm detector with Gemini API key"""
        self.api_key = api_key
        self.cv = CountVectorizer()
        self.model = BernoulliNB()
        self.gemini_model = None
        self._setup_gemini()

    def _setup_gemini(self):
        """Configure Gemini API"""
        try:
            genai.configure(api_key=self.api_key)
            self.gemini_model = genai.GenerativeModel('gemini-2.0-flash-exp')
        except Exception as e:
            print(f"Warning: Gemini API setup failed - {str(e)}")

    def load_and_train_model(self, data_path):
        """Load data and train the sarcasm detection model"""
        try:
            # Load dataset
            data = pd.read_json(data_path, lines=True)

            # Prepare data
            data = data[["headline", "is_sarcastic"]]
            x = np.array(data["headline"])
            y = np.array(data["is_sarcastic"])

            # Vectorize text data
            X = self.cv.fit_transform(x)

            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.20, random_state=42
            )

            # Train model
            self.model.fit(X_train, y_train)
            accuracy = self.model.score(X_test, y_test)

            print(f"Model Accuracy: {accuracy:.4f}")
            return True

        except Exception as e:
            print(f"Error loading data or training model: {str(e)}")
            return False

    def _get_ai_explanation(self, text, is_sarcastic, context=None):
        """Generate AI-powered explanation for sarcasm detection"""
        if not self.gemini_model:
            return "AI explanation unavailable - Gemini API not configured properly."

        try:
            if context:
                prompt = f"""
                Analyze this conversation for sarcasm detection:

                Original Message: "{context}"
                Response Received: "{text}"

                Classification Result: {'SARCASTIC' if is_sarcastic else 'NOT SARCASTIC'}

                Provide a professional analysis in 3-4 sentences explaining why this response {'is' if is_sarcastic else 'is not'} sarcastic, considering:
                - The context of the original message
                - Language patterns and tone
                - Irony or contradictions
                - Cultural/social implications
                """
            else:
                prompt = f"""
                Analyze this text for sarcasm detection:

                Text: "{text}"
                Classification Result: {'SARCASTIC' if is_sarcastic else 'NOT SARCASTIC'}

                Provide a professional analysis in 3-4 sentences explaining why this text {'is' if is_sarcastic else 'is not'} sarcastic, focusing on:
                - Linguistic patterns and word choice
                - Tone and context clues
                - Irony or implied meaning
                - Semantic contradictions
                """

            response = self.gemini_model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            return f"AI analysis temporarily unavailable: {str(e)}"

    def predict_sarcasm(self, text):
        """Predict if text is sarcastic"""
        try:
            data_transform = self.cv.transform([text]).toarray()
            prediction = self.model.predict(data_transform)[0]
            confidence = self.model.predict_proba(data_transform)[0]
            return prediction, max(confidence)
        except Exception as e:
            print(f"Prediction error: {str(e)}")
            return None, None

    def analyze_single_text(self):
        """Option 1: Analyze single text input"""
        print("\n" + "="*60)
        print("📝 SINGLE TEXT ANALYSIS")
        print("="*60)

        user_text = input("\nEnter text to analyze: ").strip()

        if not user_text:
            print("❌ Please enter valid text.")
            return

        print("\n🔄 Analyzing...")

        prediction, confidence = self.predict_sarcasm(user_text)

        if prediction is None:
            print("❌ Analysis failed. Please try again.")
            return

        # Display results
        result = "SARCASTIC" if prediction == 1 else "NOT SARCASTIC"
        print(f"\n📊 ANALYSIS RESULTS")
        print("-" * 30)
        print(f"Classification: {result}")
        print(f"Confidence: {confidence:.2%}")

        print(f"\n🤖 AI EXPLANATION:")
        print("-" * 30)
        explanation = self._get_ai_explanation(user_text, prediction == 1)
        print(explanation)

    def analyze_conversation(self):
        """Option 2: Analyze conversation context"""
        print("\n" + "="*60)
        print("💬 CONVERSATION ANALYSIS")
        print("="*60)

        original_msg = input("\nEnter your original message: ").strip()
        response_msg = input("Enter the response you received: ").strip()

        if not original_msg or not response_msg:
            print("❌ Please enter both messages.")
            return

        print("\n🔄 Analyzing conversation...")

        prediction, confidence = self.predict_sarcasm(response_msg)

        if prediction is None:
            print("❌ Analysis failed. Please try again.")
            return

        # Display results
        result = "SARCASTIC" if prediction == 1 else "NOT SARCASTIC"
        print(f"\n📊 ANALYSIS RESULTS")
        print("-" * 30)
        print(f"Response Classification: {result}")
        print(f"Confidence: {confidence:.2%}")

        print(f"\n🤖 CONTEXTUAL AI EXPLANATION:")
        print("-" * 30)
        explanation = self._get_ai_explanation(response_msg, prediction == 1, original_msg)
        print(explanation)

    def display_menu(self):
        """Display main menu options"""
        print("\n" + "="*60)
        print("MENU OPTIONS")
        print("="*60)
        print("1️⃣  Single Text Analysis")
        print("2️⃣  Conversation Analysis")
        print("3️⃣  Exit System")
        print("="*60)

    def run(self):
        """Main application loop"""
        print("\n🔍 SARCASM DETECTION SYSTEM")
        print("="*60)
        print("AI-Powered Sarcasm Analysis | Professional Edition")
        print("="*60)

        while True:
            self.display_menu()
            choice = input("\nSelect option (1-3): ").strip()

            if choice == "1":
                self.analyze_single_text()
            elif choice == "2":
                self.analyze_conversation()
            elif choice == "3":
                print("\n✅ Thank you for using the Sarcasm Detection System!")
                print("👋 Session ended successfully.")
                break
            else:
                print("❌ Invalid selection. Please choose 1, 2, or 3.")

            input("\nPress Enter to continue...")

def main():
    """Initialize and run the application"""
    # Configuration
    GEMINI_API_KEY = ""  # Replace with your actual API key
    DATA_PATH = "/content/Sarcasm.json"

    # Initialize detector
    detector = SarcasmDetector(GEMINI_API_KEY)

    # Load and train model
    print("🚀 Initializing Sarcasm Detection System...")
    print("📊 Loading dataset and training model...")

    if detector.load_and_train_model(DATA_PATH):
        detector.run()
    else:
        print("❌ Failed to initialize system. Please check your data file.")

if __name__ == "__main__":
    main()

🚀 Initializing Sarcasm Detection System...
📊 Loading dataset and training model...
Model Accuracy: 0.8448

🔍 SARCASM DETECTION SYSTEM
AI-Powered Sarcasm Analysis | Professional Edition

MENU OPTIONS
1️⃣  Single Text Analysis
2️⃣  Conversation Analysis
3️⃣  Exit System

Select option (1-3): 1

📝 SINGLE TEXT ANALYSIS

Enter text to analyze: "I love being stuck in traffic for two hours."

🔄 Analyzing...

📊 ANALYSIS RESULTS
------------------------------
Classification: SARCASTIC
Confidence: 65.17%

🤖 AI EXPLANATION:
------------------------------
The text is classified as sarcastic due to the stark semantic contradiction between the positive sentiment expressed ("I love") and the universally negative experience described ("being stuck in traffic for two hours"). The statement lacks genuine positive tone and context usually associated with expressing enjoyment, highlighting the speaker's frustration instead. This irony arises because the intended meaning is the opposite of the literal inte